# Object Detection Demo
Welcome to the object detection inference walkthrough!  This notebook will walk you step by step through the process of using a pre-trained model to detect objects in an image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

 vbn,./# Imports

In [4]:
 import tensorflow as tf

In [5]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')


## Env setup

In [6]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [7]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing `PATH_TO_FROZEN_GRAPH` to point to a new .pb file.  

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

In [8]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

## Download Model

In [9]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

## Load a (frozen) Tensorflow model into memory.

In [10]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

## Helper code

In [12]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [13]:
import pathlib
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('C:/Users/Bunny/Documents/TensorFlow/models/research/object_detection/test_images')
        
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jfif")))
        

IMAGE_SIZE = (12,8)
TEST_IMAGE_PATHS

[WindowsPath('C:/Users/Bunny/Documents/TensorFlow/models/research/object_detection/test_images/fe357170-0b5f-427b-9a47-faaa165e725c.jfif')]

In [19]:
import cv2,time
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    sess.run(tf.global_variables_initializer())
    img = 1
    for image_path in TEST_IMAGE_PATHS:
      image = Image.open(image_path)
      image_np = load_image_into_numpy_array(image)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
      
      
    
    
      (boxes, scores, classes, num_detections) = sess.run(
          [boxes, scores, classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})

      out = vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8,
          min_score_thresh=0.1)
       
      
      #ymin = int((boxes[0][0][0]*image_height))
      #xmin = int((boxes[0][0][1]*image_width))
      #ymax = int((boxes[0][0][2]*image_height))
      #xmax = int((boxes[0][0][3]*image_width))
      #(left, right, top, bottom) = (xmin * image_width, xmax * image_width, ymin * image_height, ymax * image_height)
      #Result = np.array(image_np[ymin:ymax,xmin:xmax])
      #img_item = "C:/Users/Bunny/Documents/TensorFlow/models/research/object_detection/test_images"
      #cv2.imwrite(img_item, Result)
      
      #get objects name from image having score value greater then 50%)
      your_list = [category_index.get(value) for index,value in enumerate(classes[0]) if scores[0,index] > 0.1]
      name = [item['name'] for item in your_list] #names of objects in image
      print(name)              #print out names i.e used in making decription
      var_len = len(name)         #lenght of objects , used while making description #this variable is used in description to tell number of objects 
      

      im_width, im_height = IMAGE_SIZE
      coordinates_list = []
      result = []  
      counter_for = 0
      for i,b in enumerate(boxes[0]):
          if scores[0,i] > 0.1:
             ymin = int((boxes[0][i][0]*im_height))  #top
             xmin = int((boxes[0][i][1]*im_width))   #left
             ymax = int((boxes[0][i][2]*im_height))  #bottom
             xmax = int((boxes[0][i][3]*im_width))    #right
             (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height) 
             coordinates_list.append([ymin, ymax, xmin, xmax])
             counter_for = counter_for + 1
             result.append([(xmin + xmax) / 2])
             #middle_col = (xmin + xmax) / 2
             #print(middle_row,middle_col)
             
                      
                    
      print(coordinates_list)
      print(result)   
      plt.figure(figsize=IMAGE_SIZE)
      plt.imshow(image_np)
      print(var_len)
      
      
    
    
    #copy paste this cell again

['car', 'umbrella', 'person', 'person', 'umbrella', 'car', 'person', 'person', 'person', 'car', 'car']
[[4, 4, 7, 7], [4, 4, 5, 5], [5, 5, 4, 4], [5, 6, 9, 10], [4, 4, 5, 6], [4, 4, 7, 7], [5, 5, 4, 4], [5, 5, 3, 3], [5, 6, 1, 1], [4, 4, 7, 8], [5, 5, 0, 1]]
[[7.0], [5.0], [4.0], [9.5], [5.5], [7.0], [4.0], [3.0], [1.0], [7.5], [0.5]]
11


In [52]:
print(result)
print(name)
zipped = zip(result, name)
list(zipped)

  
          


[[7.0], [5.0], [4.0], [9.5], [5.5], [7.0], [4.0], [3.0], [1.0], [7.5], [0.5]]
['car', 'umbrella', 'person', 'person', 'umbrella', 'car', 'person', 'person', 'person', 'car', 'car']


[([7.0], 'car'),
 ([5.0], 'umbrella'),
 ([4.0], 'person'),
 ([9.5], 'person'),
 ([5.5], 'umbrella'),
 ([7.0], 'car'),
 ([4.0], 'person'),
 ([3.0], 'person'),
 ([1.0], 'person'),
 ([7.5], 'car'),
 ([0.5], 'car')]

In [103]:
dictOfElems = dict()
counter1 = 0
l = []
for b,n in zip(result,name):
    if b >= [0.0] and  b <= [4.0]:
        left_obj = "  the " + n + " is on left side of image" 
        print(left_obj)
           
    elif b > [4.0]  and b <= [8.0]:
        cen_obj = "  the " + n + " is on center side of image"
        print(cen_obj)
           
    elif b > [8.0]  and b <= [12.0]: 
        right_obj = "  the " + n + " is on right side of image"
        print(right_obj)
           
    else:
        print(' ') 

  the car is on center side of image
  the umbrella is on center side of image
  the person is on left side of image
  the person is on right side of image
  the umbrella is on center side of image
  the car is on center side of image
  the person is on left side of image
  the person is on left side of image
  the person is on left side of image
  the car is on center side of image
  the car is on left side of image


In [54]:
#here putting objects that are in right, left and center, into seperate list i.e left_item_list)  and from this further will count the deplicate object in each  list i.e 2 person, 3 car
left_item_list = []
center_item_list =[]
right_item_list =[]
for b,n in zip(result,name):
    if b >= [0.0] and  b <= [4.0]:
        left_item_list.append(n) 
    elif b > [4.0]  and b <= [8.0]:  
        center_item_list.append(n)
    elif b > [8.0]  and b <= [12.0]:
         right_item_list.append(n)
print(left_item_list)        
print(center_item_list)         
print(right_item_list)     

['person', 'person', 'person', 'person', 'car']
['car', 'umbrella', 'umbrella', 'car', 'car']
['person']


In [95]:
# this cell is basically for counting deplicate object in complete image 
print(name) 

def getDuplicatesWithCount(name):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfElems = dict()
    # Iterate over each element in list
    for elem in name:
        # If element exists in dict then increment its value else add it in dict
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
 
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
    dictOfElems = { key:value for key, value in dictOfElems.items() if value > 1}
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfElems

# Get a dictionary containing duplicate elements in list and their frequency count
dictOfElems = getDuplicatesWithCount(name)     
total_obj_count = [] 
for key, value in dictOfElems.items():
          co_item = key , ' :: ', value
          total_obj_count.append(str(value) + ' are ' + str(key) + ', ')
    
print(total_obj_count)
        

['car', 'umbrella', 'person', 'person', 'umbrella', 'car', 'person', 'person', 'person', 'car', 'car']
['4 are car, ', '2 are umbrella, ', '5 are person, ']


In [56]:
#this is for counting deplicate object ob left side of image 
def getDuplicatesWithCount(left_item_list):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfElems = dict()
    # Iterate over each element in list
    for elem in left_item_list:
        # If element exists in dict then increment its value else add it in dict
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
 
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
    dictOfElems = { key:value for key, value in dictOfElems.items() if value >= 1}
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfElems

# Get a dictionary containing duplicate elements in list and their frequency count
dictOfElems = getDuplicatesWithCount(left_item_list)     
left_count_obj = [] 
for key, value in dictOfElems.items():
          co_item = key , ' :: ', value
          left_count_obj.append(str(value) + ' ' + str(key) + '')
          
          print(left_count_obj)

['4 person']
['4 person', '1 car']


In [57]:
#this is for counting deplicate object ob center side of image 
def getDuplicatesWithCount(center_item_list):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfElems = dict()
    # Iterate over each element in list
    for elem in center_item_list:
        # If element exists in dict then increment its value else add it in dict
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
 
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
    dictOfElems = { key:value for key, value in dictOfElems.items() if value >= 1}
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfElems

# Get a dictionary containing duplicate elements in list and their frequency count
dictOfElems = getDuplicatesWithCount(center_item_list)     
center_count_obj = [] 
for key, value in dictOfElems.items():
          co_item = key , ' :: ', value
          center_count_obj.append(str(value) + ' ' + str(key) + '')
            
          print(center_count_obj)

['3 car']
['3 car', '2 umbrella']


In [58]:
#this is for counting deplicate object ob right side of image 
def getDuplicatesWithCount(right_item_list):
    ''' Get frequency count of duplicate elements in the given list '''
    dictOfElems = dict()
    # Iterate over each element in list
    for elem in right_item_list:
        # If element exists in dict then increment its value else add it in dict
        if elem in dictOfElems:
            dictOfElems[elem] += 1
        else:
            dictOfElems[elem] = 1    
 
    # Filter key-value pairs in dictionary. Keep pairs whose value is greater than 1 i.e. only duplicate elements from list.
    dictOfElems = { key:value for key, value in dictOfElems.items() if value >= 1}
    # Returns a dict of duplicate elements and thier frequency count
    return dictOfElems

# Get a dictionary containing duplicate elements in list and their frequency count
dictOfElems = getDuplicatesWithCount(right_item_list)     
right_count_obj = [] 
for key, value in dictOfElems.items():
          co_item = key , ' :: ', value
          right_count_obj.append(str(value) + ' ' + str(key) + '')
        
          print(right_count_obj)
         

['1 person']


In [100]:
if var_len == 0:
    a = 'there is no '
    
elif var_len > 0:
     a = 'there are almost ' + str(var_len) + ' '
     

des =  " " + str(a) + " objects identified in given image, among them "  + '' .join(total_obj_count) + " moreover " + '' .join(left_count_obj)  + " at left side and, " + ''.join(center_count_obj) + ' are at center and  ' + '' .join(right_count_obj) + " are at right side of image" 
print(des)
   

 there are almost 11  objects identified in given image, among them 4 are car, 2 are umbrella, 5 are person,  moreover 4 person1 car at left side and, 3 car2 umbrella are at center and  1 person are at right side of image


there are almost 11 objects identified in given image
